In [289]:
import torch
import torch.nn as nn
# !pip install easyfsl
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Omniglot
from torchvision.models import resnet18
from tqdm import tqdm

from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average

class PrototypicalNetworks_dynamic_query(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks_dynamic_query, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images,
        support_labels,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        """
        num_classes=len(list(support_labels))
        prototypes = []
        for class_label in range(num_classes):
            # print(type(support_images[class_label]),torch.cat(support_images[class_label].shape)
            support_images_eachclass=(support_images[class_label])
            # print("kk", support_images_eachclass.shape)
            class_features = self.backbone(support_images_eachclass.to(device))  # Select features for the current class
            class_prototype = class_features.mean(dim=0)  # Compute the mean along the batch dimension
            prototypes.append(class_prototype)
        prototypes = torch.stack(prototypes)


        distances=[]
        for each_query_class in query_images:

          query_features = self.backbone(each_query_class)

          # Compute the distance between query features and prototypes
          distance = torch.cdist(query_features, prototypes)
          distances.append(-distance)

        # print(distances.shape)

        return distances


In [290]:
!ls '/home/asufian/Desktop/output_olchiki/code/MAPI Mayee'

'MAPI Mayeek'


In [291]:
root_path = '/home/asufian/Desktop/output_olchiki/code/MAPI Mayee'

In [292]:
import os
import random
from collections import defaultdict
from PIL import Image, ImageOps
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class PrototypicalOmniglotDataset(Dataset):
    def __init__(self, root, num_classes=1623, n_shot=5, n_query=10, transform=None):
        self.root = root
        self.num_classes = num_classes
        self.n_shot = n_shot
        self.n_query = n_query
        self.transform = transform
        self.samples_by_label = defaultdict(list)
        self.all_imgs = {}
        self.classes = []

        # Common image file extensions
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.webp']

        # Organize data into classes and image paths
        for alphabet in os.listdir(self.root):
            alphabet_path = os.path.join(self.root, alphabet)
            if not os.path.isdir(alphabet_path):
                continue

            for char_class in os.listdir(alphabet_path):
                char_class_path = os.path.join(alphabet_path, char_class)

                # Skip if the entry is not a directory
                if not os.path.isdir(char_class_path):
                    continue

                all_images = []
                for img_name in sorted(os.listdir(char_class_path)):
                    img_path = os.path.join(char_class_path, img_name)

                    # Check if it's a file with a recognized image extension
                    if os.path.isfile(img_path) and any(img_name.lower().endswith(ext) for ext in image_extensions):
                        all_images.append(img_path)

                if all_images:
                    char_class_name = f"{alphabet}_{char_class}"
                    self.samples_by_label[char_class_name] = list(range(len(all_images)))
                    self.all_imgs[char_class_name] = all_images
                    self.classes.append(char_class_name)
                    # print(char_class_name)

    # def transform_image(self, raw_img):
    #     img = ImageOps.invert(raw_img)
    #     if self.transform is not None:
    #         img = self.transform(img)
    #     return img
    def transform_image(self, raw_img):
        img = raw_img#ImageOps.invert(raw_img)
        if self.transform is not None:
            # print(self.transform)
            img = self.transform(img)
        # print(img.shape)
        return img

    def __getitem__(self, selected_classes,selected_supports):#
        # selected_classes = random.sample(self.classes, self.num_classes)
        class_indices = [self.samples_by_label[each_cls] for each_cls in selected_classes]
        # print("selected_classes: ",selected_classes)
        support_set = []
        query_set = []
        label_id = 0
        qs=[]
        ko=0
        ghj=[]
        for idx_set in class_indices:
            # Creating support set
            selected_support =selected_supports[ko]# random.sample(idx_set, self.n_shot)
            ko=ko+1
            ghj.append(selected_support)
            # print("selected_support images: ", ghj)
            support_images = [self.transform_image(Image.open(self.all_imgs[selected_classes[label_id]][each]).convert('L')) for each in selected_support]

            # Creating query set
            selected_query = [item for item in idx_set if item not in selected_support]
            query_images = [self.transform_image(Image.open(self.all_imgs[selected_classes[label_id]][each]).convert('L')) for each in selected_query]
            support_set.append((support_images, [label_id for _ in range(self.n_shot)]))
            # print("aaaaaaaaaa",label_id)
            query_set.append((query_images,  [label_id for _ in range(len(selected_query))]))

            label_id += 1
            # qs= [item for item in a for _ in range(self.n_query)]
        return support_set, query_set

    def __len__(self):
        return 10

# Set up the transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
])

# Set up the SiameseOmniglotDataset
# Training == evaluation
# root_path = './assamese'
num_classes = 5  # Set the desired number of classes per episode
n_shot = 5
n_query = 10

PrototypicalOmniglotDatasetLoader = PrototypicalOmniglotDataset(root=root_path, num_classes=num_classes, n_shot=n_shot, n_query=n_query, transform=transform)


In [293]:
import torch
import torch.nn as nn

class PrototypicalNetworks_dynamic_query(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks_dynamic_query, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images,
        support_labels,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        """
        num_classes=len(list(support_labels))
        prototypes = []
        for class_label in range(num_classes):
            # print(type(support_images[class_label]),torch.cat(support_images[class_label].shape)
            support_images_eachclass=(support_images[class_label])
            # print("kk", support_images_eachclass.shape)
            class_features = self.backbone(support_images_eachclass.to(device))  # Select features for the current class
            class_prototype = class_features.mean(dim=0)  # Compute the mean along the batch dimension
            prototypes.append(class_prototype)
        prototypes = torch.stack(prototypes)


        distances=[]
        for each_query_class in query_images:

          query_features = self.backbone(each_query_class)

          # Compute the distance between query features and prototypes
          distance = torch.cdist(query_features, prototypes)
          distances.append(-distance)

        # print(distances.shape)

        return distances


In [294]:
class PrototypicalNetworks33(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks33, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images,
        support_labels,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        num_classes = len(support_labels)
        prototypes = []
        for class_label in range(num_classes):
            support_images_eachclass = support_images[class_label]
            class_features = torch.stack([self.backbone(each.unsqueeze(0)) for each in support_images_eachclass])
            prototype_bymean = class_features.mean(dim=0)
            # print(prototype_bymean.shape)
            prototypes.append(prototype_bymean)

        query_features = torch.stack([self.backbone(each.unsqueeze(0)) for each in query_images])
        # print(query_features.shape,query_features[0].shape,torch.stack(prototypes).shape,torch.stack(prototypes)[0].shape)
        # distances = -torch.cdist(query_features, torch.stack(prototypes))
        distances = []  # Initialize list to store distances
        prototypes=torch.stack(prototypes)
        for each_query_feature in query_features:
            d = torch.cdist(prototypes, each_query_feature)  # Compute distances
            distances.append(d.squeeze())  # Remove singleton dimension and store
            # print(d.shape)  # Output: [5, 1], shape of each distance matrix

        distances = torch.stack(distances)

        return -distances


In [296]:
import pandas as pd
import os
import csv
def save_in_csv(cls,flnm,conf_matrix, acciuracy,precision_overall, recall_overall, f_betas):
  with open(flnm, 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      headers=['Accuracy', 'Precision', 'Recall']
      beta=-5
      values=[]
      values.append(acciuracy)
      values.append(precision_overall)
      values.append(recall_overall)
      for each in f_betas:
              headers.append('F-betas (' +str(beta)+' )')
              beta=beta+1
              values.append(each)

    # Write confusion matrix
      writer.writerow(['used Class:'])
      writer.writerows([cls])
      writer.writerow([])
      writer.writerow([])
      writer.writerow(['Confusion Matrix'])
      writer.writerows(conf_matrix)
      writer.writerow([])
      writer.writerow([])  # Add an empty row for separation

      # Write metrics

      writer.writerow(headers)
      writer.writerow(values)


  # print("Metrics have been written to 'metrics.csv' file.")
def save_in_text_file(msg,cls, flnm, conf_matrix, accuracy, precision_overall, recall_overall, f_betas):
    with open(flnm, 'a') as textfile:
        textfile.write(str("\n\n\n                                 "+msg))
        textfile.write("\n\n\n")
        textfile.write("used Class:\n")
        textfile.write(str(cls) + "\n\n")

        textfile.write("Confusion Matrix:\n")
        for row in conf_matrix:
            textfile.write(' '.join([str(elem) for elem in row]) + "\n")
        textfile.write("\n")

        textfile.write("Metrics:\n")
        textfile.write("Accuracy: {}\n".format(accuracy))
        textfile.write("Precision: {}\n".format(precision_overall))
        textfile.write("Recall: {}\n".format(recall_overall))

        beta = -5
        for f_beta in f_betas:
            textfile.write("F-betas ({}): {}\n".format(beta, f_beta))
            beta += 1

    # print("Metrics have been written to '{}' file.".format(flnm))

def save_in_excel_final(msg, cls, flnm, conf_matrix, accuracy, precision_overall, recall_overall):
    data = {
        "Used Class": [cls],
        "Used support images idx": [msg],
        "Confusion Matrix": [conf_matrix],
        "Accuracy": [accuracy],
        "Precision": [precision_overall],
        "Recall": [recall_overall]
    }
    # for i, f_beta in enumerate(f_betas):
    #     data[f"F-beta ({i - 5})"] = [f_beta]

    df_new = pd.DataFrame(data)
    df_new = df_new.transpose()  # Transpose the DataFrame

    if os.path.exists(flnm):
        df_existing = pd.read_excel(flnm, index_col=0)  # Read existing file
        df_combined = pd.concat([df_existing, df_new], axis=1)  # Concatenate along columns
        df_combined.to_excel(flnm)  # Write to Excel
    else:
        df_new.to_excel(flnm)  # Write new DataFrame to Excel

    # print("Metrics have been written to '{}' file.".format(flnm))


In [297]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch
import torch.nn as nn
import torch.hub

# Define a modified version of DenseNet169 with dropout after each convolutional layer
class DenseNet169WithDropout(nn.Module):
    def __init__(self,modell, pretrained=False, dr=0.4):
        super(DenseNet169WithDropout, self).__init__()
        self.densenet169 = torch.hub.load('pytorch/vision:v0.10.0', modell, pretrained=pretrained)
        self.dropout = nn.Dropout(p=dr)  # Dropout with probability 0.5

        # Modify each dense block to include dropout after each convolutional layer
        for name, module in self.densenet169.features.named_children():
            if isinstance(module, nn.Sequential):
                for sub_name, sub_module in module.named_children():
                    if isinstance(sub_module, nn.Conv2d):
                        setattr(module, sub_name, nn.Sequential(sub_module, self.dropout))

    def forward(self, x):
        return self.densenet169(x)

# Instantiate the modified DenseNet169 model with dropout
# convolutional_network_with_dropout = DenseNet169WithDropout(modell='resnet18',pretrained=False,dr=0.4)

# Example usage:
# output = convolutional_network_with_dropout(input_tensor)

class ResNet18WithDropout(nn.Module):
    def __init__(self, pretrained=False, dr=0.4):
        super(ResNet18WithDropout, self).__init__()
        self.resnet18 = models.resnet18(pretrained=pretrained)
        self.dropout = nn.Dropout(p=dr)

        # Modify each residual block to include dropout after each convolutional layer
        for name, module in self.resnet18.named_children():
            if isinstance(module, nn.Sequential):
                for sub_name, sub_module in module.named_children():
                    if isinstance(sub_module, nn.Conv2d):
                        setattr(module, sub_name, nn.Sequential(sub_module, self.dropout))

    def forward(self, x):
        return self.resnet18(x)

# Instantiate the modified ResNet18 model with dropout
# convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.4)


In [449]:


clss_5=[
     ['MAPI Mayeek_GHOU', 'MAPI Mayeek_JIL', 'MAPI Mayeek_SAM', 'MAPI Mayeek_KOK', 'MAPI Mayeek_MIT'],
      ['MAPI Mayeek_KOK', 'MAPI Mayeek_DIL', 'MAPI Mayeek_PAA', 'MAPI Mayeek_ATIYA', 'MAPI Mayeek_MIT'],
     ['MAPI Mayeek_DHOU', 'MAPI Mayeek_KOK', 'MAPI Mayeek_RAAI', 'MAPI Mayeek_TIL', 'MAPI Mayeek_GHOU'],
      ['MAPI Mayeek_FAM', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_LAI', 'MAPI Mayeek_PAA', 'MAPI Mayeek_KOK'],
     ['MAPI Mayeek_KOK', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_MIT', 'MAPI Mayeek_WAI'],
['MAPI Mayeek_KOK', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_MIT', 'MAPI Mayeek_WAI'],
     ['MAPI Mayeek_NAA', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_BAA', 'MAPI Mayeek_LAI', 'MAPI Mayeek_RAAI'],
     ['MAPI Mayeek_RAAI', 'MAPI Mayeek_FAM', 'MAPI Mayeek_GOK', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_SAM'],
     ['MAPI Mayeek_EEE', 'MAPI Mayeek_KOK', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_WAI'],
     ['MAPI Mayeek_DHOU', 'MAPI Mayeek_RAAI', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_UUN', 'MAPI Mayeek_LAI'],
     ['MAPI Mayeek_HUK', 'MAPI Mayeek_FAM', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_THOU', 'MAPI Mayeek_SAM'],
     ['MAPI Mayeek_MIT', 'MAPI Mayeek_LAI', 'MAPI Mayeek_DIL', 'MAPI Mayeek_ATIYA', 'MAPI Mayeek_BHAM'],
     ['MAPI Mayeek_NGOU', 'MAPI Mayeek_FAM', 'MAPI Mayeek_ATIYA', 'MAPI Mayeek_MIT', 'MAPI Mayeek_JHAM'],
     ['MAPI Mayeek_GOK', 'MAPI Mayeek_ATIYA', 'MAPI Mayeek_LAI', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_KHOU'],
['MAPI Mayeek_HUK', 'MAPI Mayeek_DIL', 'MAPI Mayeek_KOK', 'MAPI Mayeek_PAA', 'MAPI Mayeek_LAI']

     ]

clss_support_imagesss_1_shot=[[[3], [56], [93], [62], [23]],
             [[34], [18], [31], [40], [39]],
             [[54], [61], [9], [50], [93]],
             [[85], [9], [35], [6], [36]],
             [[93], [87], [60], [26], [35]],
             [[2], [39], [84], [38], [30]],
             [[6], [12], [43], [65], [66]],
             [[6], [66], [6], [13], [89]],
             [[50], [70], [13], [61], [7]],
             [[54], [58], [26], [75], [51]],
             [[14], [27], [92], [76], [27]],
             [[47], [34], [84], [45], [82]],
             [[63], [69], [25], [80], [61]],
             [[13], [41], [57], [13], [46]],
             [[58], [92], [48], [92], [35]]
]

clss_support_imagesss_5_shot=[[[3, 21, 2, 72, 87], [56, 22, 58, 3, 24], [93, 76, 48, 81, 68], [62, 82, 2, 87, 69], [23, 73, 83, 92, 55]], [[34, 22, 25, 30, 74], [18, 70, 45, 20, 46], [31, 83, 42, 49, 14], [40, 87, 93, 12, 92], [39, 20, 70, 74, 84]], [[54, 11, 2, 80, 8], [61, 72, 52, 17, 95], [9, 2, 37, 15, 26], [50, 48, 2, 79, 63], [93, 48, 0, 24, 89]], [[85, 19, 68, 55, 56], [9, 42, 35, 93, 43], [35, 29, 91, 2, 68], [6, 56, 30, 50, 25], [36, 42, 19, 21, 41]], [[93, 9, 38, 61, 68], [87, 33, 27, 36, 53], [60, 21, 53, 49, 17], [26, 2, 11, 62, 4], [35, 78, 30, 79, 84]], [[2, 70, 12, 56, 5], [39, 59, 30, 45, 95], [84, 86, 14, 66, 54], [38, 54, 9, 46, 34], [30, 72, 47, 6, 26]], [[6, 1, 61, 70, 8], [12, 33, 77, 55, 51], [43, 56, 94, 74, 45], [65, 27, 13, 11, 6], [66, 93, 36, 68, 27]], [[6, 3, 81, 24, 20], [66, 73, 68, 55, 76], [6, 14, 55, 3, 38], [13, 86, 45, 89, 18], [89, 65, 74, 8, 42]], [[50, 91, 0, 7, 51], [70, 50, 17, 33, 64], [13, 32, 8, 38, 72], [61, 43, 74, 86, 95], [7, 34, 70, 20, 36]], [[54, 61, 75, 15, 50], [58, 7, 51, 93, 56], [26, 89, 91, 57, 24], [75, 49, 33, 88, 50], [51, 92, 53, 54, 52]], [[14, 80, 78, 27, 3], [27, 69, 59, 63, 93], [92, 60, 56, 86, 64], [76, 30, 64, 49, 12], [27, 30, 35, 64, 56]], [[47, 94, 69, 51, 5], [34, 28, 59, 14, 67], [84, 34, 23, 5, 16], [45, 0, 57, 88, 76], [82, 78, 30, 12, 49]], [[63, 89, 62, 51, 88], [69, 75, 83, 49, 45], [25, 71, 45, 26, 72], [80, 92, 22, 41, 62], [61, 25, 40, 14, 94]], [[13, 82, 90, 45, 14], [41, 60, 79, 40, 67], [57, 94, 27, 77, 5], [13, 24, 54, 1, 94], [46, 62, 69, 2, 85]], [[58, 5, 69, 35, 30], [92, 88, 79, 58, 31], [48, 43, 78, 9, 15], [92, 87, 34, 6, 75], [35, 16, 4, 21, 46]]]

clss_support_imagesss_10_shot=[[[3, 21, 2, 72, 87, 60, 81, 24, 28, 0], [56, 22, 58, 3, 24, 6, 89, 25, 59, 39], [93, 76, 48, 81, 68, 39, 40, 17, 58, 89], [62, 82, 2, 87, 69, 65, 4, 35, 9, 59], [23, 73, 83, 92, 55, 14, 82, 38, 6, 70]], [[34, 22, 25, 30, 74, 12, 26, 79, 63, 78], [18, 70, 45, 20, 46, 77, 23, 35, 29, 22], [31, 83, 42, 49, 14, 74, 32, 41, 72, 55], [40, 87, 93, 12, 92, 32, 73, 16, 44, 83], [39, 20, 70, 74, 84, 73, 87, 88, 66, 54]], [[54, 11, 2, 80, 8, 93, 75, 35, 86, 50], [61, 72, 52, 17, 95, 26, 46, 64, 88, 40], [9, 2, 37, 15, 26, 59, 32, 24, 83, 51], [50, 48, 2, 79, 63, 42, 64, 81, 72, 58], [93, 48, 0, 24, 89, 74, 42, 88, 71, 35]], [[85, 19, 68, 55, 56, 14, 93, 49, 90, 9], [9, 42, 35, 93, 43, 89, 4, 92, 84, 33], [35, 29, 91, 2, 68, 51, 52, 39, 59, 85], [6, 56, 30, 50, 25, 38, 85, 84, 93, 47], [36, 42, 19, 21, 41, 72, 62, 86, 25, 18]], [[93, 9, 38, 61, 68, 22, 48, 77, 45, 76], [87, 33, 27, 36, 53, 77, 90, 93, 39, 9], [60, 21, 53, 49, 17, 41, 36, 81, 63, 7], [26, 2, 11, 62, 4, 3, 66, 82, 7, 0], [35, 78, 30, 79, 84, 67, 65, 19, 66, 41]], [[2, 70, 12, 56, 5, 1, 76, 53, 69, 73], [39, 59, 30, 45, 95, 60, 85, 62, 70, 67], [84, 86, 14, 66, 54, 24, 63, 52, 93, 49], [38, 54, 9, 46, 34, 84, 8, 35, 7, 55], [30, 72, 47, 6, 26, 48, 13, 67, 19, 52]], [[6, 1, 61, 70, 8, 79, 46, 62, 85, 95], [12, 33, 77, 55, 51, 0, 19, 42, 85, 69], [43, 56, 94, 74, 45, 95, 29, 6, 0, 4], [65, 27, 13, 11, 6, 53, 16, 5, 4, 47], [66, 93, 36, 68, 27, 86, 60, 70, 58, 57]], [[6, 3, 81, 24, 20, 59, 32, 73, 91, 22], [66, 73, 68, 55, 76, 41, 91, 88, 59, 74], [6, 14, 55, 3, 38, 19, 85, 34, 62, 10], [13, 86, 45, 89, 18, 22, 59, 11, 73, 72], [89, 65, 74, 8, 42, 85, 48, 29, 50, 13]], [[50, 91, 0, 7, 51, 19, 11, 78, 59, 13], [70, 50, 17, 33, 64, 92, 94, 84, 54, 81], [13, 32, 8, 38, 72, 51, 66, 85, 62, 74], [61, 43, 74, 86, 95, 80, 32, 59, 50, 77], [7, 34, 70, 20, 36, 30, 67, 9, 94, 43]], [[54, 61, 75, 15, 50, 24, 37, 84, 27, 52], [58, 7, 51, 93, 56, 38, 35, 82, 46, 50], [26, 89, 91, 57, 24, 7, 86, 12, 3, 44], [75, 49, 33, 88, 50, 82, 8, 12, 81, 56], [51, 92, 53, 54, 52, 26, 27, 74, 22, 72]], [[14, 80, 78, 27, 3, 1, 6, 32, 81, 2], [27, 69, 59, 63, 93, 3, 5, 44, 19, 94], [92, 60, 56, 86, 64, 47, 48, 24, 79, 38], [76, 30, 64, 49, 12, 15, 14, 1, 28, 21], [27, 30, 35, 64, 56, 92, 53, 0, 60, 41]], [[47, 94, 69, 51, 5, 93, 22, 78, 73, 36], [34, 28, 59, 14, 67, 48, 74, 6, 46, 10], [84, 34, 23, 5, 16, 40, 28, 92, 20, 12], [45, 0, 57, 88, 76, 69, 17, 23, 77, 61], [82, 78, 30, 12, 49, 10, 79, 2, 50, 19]], [[63, 89, 62, 51, 88, 28, 34, 64, 16, 78], [69, 75, 83, 49, 45, 19, 23, 72, 3, 0], [25, 71, 45, 26, 72, 43, 27, 41, 81, 94], [80, 92, 22, 41, 62, 87, 5, 33, 12, 49], [61, 25, 40, 14, 94, 85, 46, 67, 87, 30]], [[13, 82, 90, 45, 14, 31, 46, 18, 92, 40], [41, 60, 79, 40, 67, 92, 64, 86, 72, 13], [57, 94, 27, 77, 5, 89, 6, 12, 52, 30], [13, 24, 54, 1, 94, 40, 64, 35, 6, 51], [46, 62, 69, 2, 85, 34, 0, 28, 11, 1]], [[58, 5, 69, 35, 30, 28, 4, 22, 21, 25], [92, 88, 79, 58, 31, 25, 15, 34, 9, 38], [48, 43, 78, 9, 15, 73, 91, 65, 52, 13], [92, 87, 34, 6, 75, 44, 83, 55, 47, 29], [35, 16, 4, 21, 46, 56, 14, 59, 43, 19]]]


In [391]:
!ls /home/asufian/Desktop/output_olchiki/code/

'ammmul on olchiki table4.ipynb'
'Amul_mapi traning dens169-Copy1.ipynb'
'Amul_mapi traning dens169.ipynb'
'Amul_mapi traning resnet.ipynb'
'Amul_olchiki traning dens169.ipynb'
'Amul_olchiki traning lwcnn.ipynb'
'Amul_olchiki traning resnet18.ipynb'
'Applying_AMWL(BMWA)_on_dnsnet_(with_Bise_ratio)_on_Test_Dataset_1_Assemese_Script_(1) (1).ipynb'
'Applying_AMWL(BMWA)_on_LWCNN_(with Bise ratio)_on_Test_Dataset_1_Assemese_Script.ipynb'
 asamese
 asamese_sce
'assameseAshish (1).zip'
'MAPI Mayee'
'MAPI Mayeek.zip'
 mapi_sce
 modelrdnet_169_55.pth
'modelr_res18_35 (1).pth'
 olchiki
'olchiki table1-Copy1.ipynb'
'olchiki table1.ipynb'
 own_80.pth
 own_ass_5_way_1_shot.txt
 own_ass_5_way_1_shot.xlsx
'Result_generatoow_nmodl_assamses_(1)(1)_(2).ipynb'
'Testing on Test Dataset:1(Assamese Script) (15 case)_ (for B-PN(resnet18&densNet169) and LWCNN-PN _togather.ipynb-Copy1.ipynb'
'Testing on Test Dataset:1(Assamese Script) (15 case)_ (for B-PN(resnet18&densNet169) and LWCNN-PN _togather.ipynb.ipynb

In [392]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!rm 'model_own_path.pt'
!rm 'model_mu_path.pt'

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/mapi_sce/Denssecondary/model_1/model_B_mapi_1-shot_dens.pth',map_location=torch.device('cpu')))
torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [393]:
# !ls '/home/asufian/Desktop/output_olchiki'

In [394]:
criterion = nn.CrossEntropyLoss()

In [395]:
root_path

'/home/asufian/Desktop/output_olchiki/code/MAPI Mayee'

In [396]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
import numpy as np
import pandas as pd
# root_path = '/content/assamese'
num_classes = 5
n_shot = 1
n_query = 14
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformt_ugh = transforms.Compose([
    transforms.Resize((64, 64)),
    # transforms.RandomResizedCrop(64),  # Randomly crop the image and resize to 224x224
    # transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
    # transforms.RandomAffine(degrees=0, translate=(0, 0), scale=(0.9, 1.1), shear=0),  # Random zoom (scaling)
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image with ImageNet statistics
])

print(f"Generated random values - num_classes: {num_classes}, n_shot: {n_shot}, n_query: {n_query}")

test_loader = PrototypicalOmniglotDataset(root=root_path, num_classes=num_classes, n_shot=n_shot, n_query=20, transform=transformt_ugh)

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    precision=0
    recall=0
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            # msg="\n\n       Way: "+str(len(clss_5[tlv_cls]))+"  Shot: "+str(len(clss_support_imagesss_1_shot[tlv_cls][0]))    +" \n\n"
            # msg=str(clss_5[tlv_cls])
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision=precision+precision_overall
            recall=recall+recall_overall
    print("---------------------->",ttlac/ttla,precision/ttla ,recall/ttla)
# %%%%%%%%%%%%%%%%%%%


pp='/home/asufian/Desktop/output_olchikifaltu'






Generated random values - num_classes: 5, n_shot: 1, n_query: 14


In [397]:
import numpy as np

conf_matrix = np.array([[100, 0, 0, 0, 0],
                        [0, 100, 0, 0, 0],
                        [0, 0, 100, 0, 0],
                        [0, 0, 0, 100, 0],
                        [0, 0, 0, 0, 100]])

def calculate_metrics_get_per(msg, conf_matrix,acciuracy, cls, flnm):
    if conf_matrix.size == 0:
        raise ValueError("Confusion matrix is empty!")

    metrics = []
    for i in range(conf_matrix.shape[0]):
        tp = conf_matrix[i, i]
        fp = np.sum(conf_matrix[:, i]) - tp
        fn = np.sum(conf_matrix[i, :]) - tp

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics.append((precision, recall, f1))

    # Compute overall scores
    precision_overall = np.mean([m[0] for m in metrics])
    recall_overall = np.mean([m[1] for m in metrics])
    f1_overall = np.mean([m[2] for m in metrics])

    # Compute overall accuracy
    total_samples = np.sum(conf_matrix)
    correct_predictions = np.sum(np.diag(conf_matrix))
    accuracy_overall = correct_predictions / total_samples if total_samples > 0 else 0
    # print(accuracy_overall)

    # Save results (Assuming these functions exist)
    save_in_excel_final(msg, cls, flnm + ".xlsx", conf_matrix, accuracy_overall, precision_overall, recall_overall)
    # save_in_text_file(msg, cls, flnm + ".txt", conf_matrix, accuracy_overall, precision_overall, recall_overall, f_betas)

    return precision_overall, recall_overall, f1_overall


# precision, recall, f_beta = calculate_metrics(conf_matrix)
# print(f'Overall Metrics for beta={beta}:')
# print(f'Overall Precision: {precision:.4f}')
# print(f'Overall Recall: {recall:.4f}')
# print(f'Overall F-beta ({f_beta}): ')
# print()


In [398]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

           
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [399]:
def update_model_weights2(model_a, model_b, conv_ratio=0.8, fc_ratio=0.8, bias_ratio=0.8):
    """Updates model_a's weights using a weighted combination of model_a and model_b."""
    for name, param in model_a.named_parameters():
        if name in model_b.state_dict():
            weight_b = model_b.state_dict()[name]
            
            if 'weight' in name:
                # Separate conditions for convolutional and fully connected layers
                if 'conv' in name:  # Convolutional layer weights
                    # print('conv')
                    param.data = conv_ratio * weight_b + (1 - conv_ratio) * param.data
                elif 'fc' in name or 'linear' in name:  # Fully connected layer weights
                    # print('fc')
                    param.data = fc_ratio * weight_b + (1 - fc_ratio) * param.data
            
            elif 'bias' in name:
                param.data = bias_ratio * weight_b + (1 - bias_ratio) * param.data

In [400]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to get model accuracy
def get_model_accuracy(b_a_r, c_a_r, f_a_r):
    try:
        print(f"Testing b_a_r = {b_a_r:.4f}, c_a_r = {c_a_r:.4f}, f_a_r = {f_a_r:.4f}")  # Debugging Output
        convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
        convolutional_network_with_dropout.fc = nn.Flatten()

        xyz = convolutional_network_with_dropout
        M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
        M3.load_state_dict(torch.load('model_own_path.pt'))

        # Load Model M2 (Reference)
        convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
        convolutional_network_with_dropout.fc = nn.Flatten()

        xyz2 = convolutional_network_with_dropout
        M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
        M2.load_state_dict(torch.load('model_mu_path.pt'))

        # Apply bias adaptation to M3 using M2
        update_model_weights2(M3, M2, conv_ratio=c_a_r, fc_ratio=f_a_r, bias_ratio=b_a_r)
        
        # Evaluate accuracy
        accuracy = evaluate3(pp+'/_mapi_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)
        
        if accuracy is None:
            raise ValueError("Accuracy computation failed. Returning worst case.")

        return accuracy  # Ensure accuracy is valid
    except Exception as e:
        print(f"Error encountered: {e}. Assigning worst-case accuracy of 0.")
        return 0.0  # Assign lowest accuracy to prevent optimization failures


In [401]:
pp

'/home/asufian/Desktop/output_olchikifaltu'

In [402]:
criterion = nn.CrossEntropyLoss()


In [403]:

# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(-1, 1.0000, name="b_a_r"),
    Real(-1, 1.0000, name="c_a_r"),
    Real(-1, 1.0000, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=80
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.5931, c_a_r = -0.6331, f_a_r = 0.5594


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.20996491228070177
Evaluated Accuracy: 0.2100
Testing b_a_r = 0.1937, c_a_r = -0.1083, f_a_r = -0.8001


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5569122807017544
Evaluated Accuracy: 0.5569
Testing b_a_r = -0.0815, c_a_r = -0.3326, f_a_r = -0.7143


...

----------------------> 0.5719298245614034
Evaluated Accuracy: 0.5719
Testing b_a_r = 0.9771, c_a_r = 0.7810, f_a_r = 0.0108


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.27466666666666667
Evaluated Accuracy: 0.2747
Testing b_a_r = -0.9853, c_a_r = 0.0115, f_a_r = 0.8900


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.608280701754386
Evaluated Accuracy: 0.6083

✅ Optimal Values:
   - b_a_r: -0.9444
   - c_a_r: -0.0444
   - f_a_r: 0.7712
📈 Highest Accuracy Achieved: 0.6177


In [404]:
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_mapii/result_densnet/base/5_w_1_s_f_20', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/secondary/5_w_1_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= -0.0088 , fc_ratio=-0.9687 , bias_ratio=-0.9673)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/amul/5_w_1_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_mapi_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.6114 ± 0.1115  0.6387 ± 0.1106  0.6114 ± 0.1115


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5169 ± 0.0811  0.5663 ± 0.0660  0.5169 ± 0.0811
---Accu-----pre----rec---------> 0.6149 ± 0.1150  0.6414 ± 0.1148  0.6149 ± 0.1150


In [284]:
!mkdir /home/asufian/Desktop/output_mapii/result_densnet/amul

mkdir: cannot create directory ‘/home/asufian/Desktop/output_mapii/result_densnet/amul’: File exists


In [21]:
!ls /home/asufian/Desktop/output_olchiki/code/mapi/model_5/model_B_mapi_1-shot_30.pth

model_B_olchiki_1-shot_20.pth  model_B_olchiki_1-shot_80.pth
model_B_olchiki_1-shot_30.pth  model_B_olchiki_1-shot_90.pth
model_B_olchiki_1-shot_50.pth


In [ ]:
########################  5   way    5  shot    #########################

In [450]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/mapi_sce/Denssecondary/model_5/model_B_mapi_5-shot_dens.pth',map_location=torch.device('cpu')))
torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [451]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

           
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

            
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6568498168498168
Evaluated Accuracy: 0.6568
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.3772893772893772
Evaluated Accuracy: 0.3773
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.32732600732600736
Evaluated Accuracy: 0.3273
Testing b_a_r = 0.0917, c_a_r = 0.0129, f_a_r = 0.9408


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.727179487179487
Evaluated Accuracy: 0.7272
Testing b_a_r = 0.0363, c_a_r = 0.0419, f_a_r = 0.0951


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
# b_a_r: 
#    - c_a_r: 
#    - f_a_r: 

#  pt=20

In [ ]:

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_mapii/result_densnet/base/5_w_5_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/secondary/5_w_5_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/amul/5_w_5_s_f", test_loader, M3, criterion)




# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_mapi_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

In [66]:
!ls /home/asufian/Desktop/output_olchiki/code/mapi/model_10

ls: cannot access '/home/asufian/Desktop/output_olchiki/code/mapi/model_10': No such file or directory


In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/mapi_sce/Denssecondary/model_10/model_B_mapi_10-shot_dens.pth',map_location=torch.device('cpu')))


torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [85]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

           
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

            
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [86]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.7158139534883721
Evaluated Accuracy: 0.7158
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.386046511627907
Evaluated Accuracy: 0.3860
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.7669767441860466
Evaluated Accuracy: 0.7670
Testing b_a_r = 0.0192, c_a_r = 0.0443, f_a_r = 0.0204


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.753953488372093
Evaluated Accuracy: 0.7540
Testing b_a_r = 0.9982, c_a_r = 0.1197, f_a_r = 0.0980


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.7651162790697674
Evaluated Accuracy: 0.7651

✅ Optimal Values:
   - b_a_r: 0.9865
   - c_a_r: 0.1116
   - f_a_r: 0.0957
📈 Highest Accuracy Achieved: 0.7681


In [87]:
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_mapii/result_densnet/base/5_w_10_s_ff', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/secondary/5_w_10_s_ff", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/amul/5_w_5_10_ff", test_loader, M3, criterion)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_mapi_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.7498 ± 0.0645  0.7585 ± 0.0624  0.7498 ± 0.0645


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.6375 ± 0.0725  0.6446 ± 0.0709  0.6375 ± 0.0725
---Accu-----pre----rec---------> 0.7681 ± 0.0561  0.7738 ± 0.0553  0.7681 ± 0.0561


In [88]:
ls /home/asufian/Desktop/output_olchikilol

ls: cannot access '/home/asufian/Desktop/output_olchikilol': No such file or directory


In [443]:
clss_8=[
    ['MAPI Mayeek_FAM', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_LAI', 'MAPI Mayeek_HUK', 'MAPI Mayeek_KOK', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_THOU', 'MAPI Mayeek_JIL'],
     ['MAPI Mayeek_SAM', 'MAPI Mayeek_HUK', 'MAPI Mayeek_KHOU', 'MAPI Mayeek_NAA', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_THOU', 'MAPI Mayeek_PAA', 'MAPI Mayeek_JHAM'],
    ['MAPI Mayeek_JHAM', 'MAPI Mayeek_NAA', 'MAPI Mayeek_GOK', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_KHOU', 'MAPI Mayeek_RAAI', 'MAPI Mayeek_DIL'],
    ['MAPI Mayeek_CHIN', 'MAPI Mayeek_UUN', 'MAPI Mayeek_YANG', 'MAPI Mayeek_KOK', 'MAPI Mayeek_MIT', 'MAPI Mayeek_LAI', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_NAA'],
    ['MAPI Mayeek_DHOU', 'MAPI Mayeek_PAA', 'MAPI Mayeek_MIT', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_EEE', 'MAPI Mayeek_LAI', 'MAPI Mayeek_UUN', 'MAPI Mayeek_SAM'],
    ['MAPI Mayeek_NAA', 'MAPI Mayeek_UUN', 'MAPI Mayeek_FAM', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_BAA', 'MAPI Mayeek_KOK', 'MAPI Mayeek_RAAI', 'MAPI Mayeek_LAI'],
    ['MAPI Mayeek_CHIN', 'MAPI Mayeek_KOK', 'MAPI Mayeek_HUK', 'MAPI Mayeek_PAA', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_LAI', 'MAPI Mayeek_FAM'],
    ['MAPI Mayeek_NGOU', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_DIL', 'MAPI Mayeek_KHOU', 'MAPI Mayeek_SAM', 'MAPI Mayeek_UUN', 'MAPI Mayeek_PAA', 'MAPI Mayeek_KOK'],
    ['MAPI Mayeek_GOK', 'MAPI Mayeek_SAM', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_CHIN', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_NAA', 'MAPI Mayeek_HUK', 'MAPI Mayeek_THOU'],
    ['MAPI Mayeek_BHAM', 'MAPI Mayeek_SAM', 'MAPI Mayeek_PAA', 'MAPI Mayeek_UUN', 'MAPI Mayeek_FAM', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_RAAI'],
     ['MAPI Mayeek_UUN', 'MAPI Mayeek_FAM', 'MAPI Mayeek_JIL', 'MAPI Mayeek_TIL', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_MIT', 'MAPI Mayeek_GOK', 'MAPI Mayeek_JHAM'],
    ['MAPI Mayeek_KOK', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_UUN', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_MIT', 'MAPI Mayeek_YANG', 'MAPI Mayeek_LAI'],
    ['MAPI Mayeek_FAM', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_LAI', 'MAPI Mayeek_KOK', 'MAPI Mayeek_CHIN', 'MAPI Mayeek_WAI', 'MAPI Mayeek_GOK'],
    ['MAPI Mayeek_UUN', 'MAPI Mayeek_YANG', 'MAPI Mayeek_FAM', 'MAPI Mayeek_NAA', 'MAPI Mayeek_KOK', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_HUK', 'MAPI Mayeek_WAI'],
    ['MAPI Mayeek_JIL', 'MAPI Mayeek_CHIN', 'MAPI Mayeek_TIL', 'MAPI Mayeek_LAI', 'MAPI Mayeek_PAA', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_MIT', 'MAPI Mayeek_YANG'],
 ]

clss_support_imagesss_10_shot=[[[32, 30, 44, 38, 25, 67, 9, 24, 47, 41], [82, 83, 48, 43, 90, 45, 62, 87, 5, 25], [7, 95, 9, 14, 63, 82, 64, 93, 61, 55], [72, 82, 69, 26, 6, 9, 39, 19, 30, 13], [25, 67, 41, 47, 95, 38, 76, 58, 87, 90], [44, 57, 84, 72, 58, 64, 93, 31, 4, 73], [7, 68, 51, 47, 63, 36, 4, 50, 94, 37], [67, 17, 31, 62, 12, 34, 81, 15, 29, 71]], [[50, 94, 95, 27, 35, 80, 62, 55, 82, 88], [74, 33, 78, 25, 11, 79, 52, 17, 12, 15], [33, 56, 67, 11, 31, 74, 88, 3, 24, 78], [71, 37, 27, 11, 65, 74, 3, 86, 80, 78], [67, 29, 87, 60, 71, 43, 40, 74, 61, 37], [16, 83, 67, 4, 50, 72, 77, 89, 69, 15], [20, 87, 71, 53, 90, 41, 63, 18, 44, 1], [14, 77, 74, 42, 27, 68, 35, 93, 81, 15]], [[30, 48, 0, 84, 3, 13, 36, 86, 38, 2], [93, 69, 13, 90, 78, 74, 20, 73, 33, 65], [6, 20, 80, 56, 33, 48, 75, 47, 18, 72], [20, 50, 8, 76, 15, 24, 80, 73, 95, 52], [54, 46, 37, 77, 38, 43, 27, 25, 3, 33], [50, 72, 23, 28, 85, 56, 17, 14, 70, 34], [8, 45, 72, 36, 93, 89, 40, 84, 54, 17], [71, 91, 30, 49, 66, 35, 84, 5, 63, 48]], [[4, 73, 59, 63, 39, 94, 24, 22, 64, 51], [14, 19, 31, 63, 9, 77, 43, 80, 33, 11], [40, 94, 59, 17, 57, 47, 49, 21, 85, 12], [29, 56, 59, 71, 78, 32, 34, 66, 83, 60], [19, 0, 83, 93, 15, 51, 26, 16, 35, 55], [51, 19, 38, 46, 5, 65, 80, 44, 71, 25], [40, 32, 95, 4, 81, 46, 24, 27, 68, 47], [47, 28, 38, 60, 59, 19, 18, 77, 89, 0]], [[53, 40, 2, 54, 68, 81, 23, 86, 94, 83], [23, 49, 47, 43, 7, 84, 62, 86, 6, 35], [12, 35, 13, 79, 86, 75, 58, 32, 88, 31], [2, 42, 85, 0, 7, 27, 93, 73, 25, 18], [78, 72, 46, 12, 1, 70, 18, 62, 14, 39], [9, 88, 77, 7, 84, 52, 32, 94, 53, 42], [56, 89, 26, 69, 17, 59, 34, 91, 58, 11], [73, 11, 22, 27, 48, 69, 92, 7, 26, 13]], [[62, 40, 87, 85, 93, 37, 52, 84, 30, 76], [55, 83, 60, 17, 19, 73, 36, 13, 77, 80], [76, 79, 90, 65, 60, 56, 73, 95, 47, 67], [65, 91, 34, 14, 10, 64, 93, 73, 2, 60], [44, 45, 50, 35, 13, 30, 6, 71, 7, 95], [82, 44, 14, 77, 19, 50, 10, 28, 29, 58], [18, 90, 85, 7, 16, 50, 6, 59, 93, 75], [74, 76, 91, 47, 63, 88, 8, 14, 64, 95]], [[19, 26, 77, 36, 52, 84, 8, 70, 90, 37], [23, 34, 84, 62, 4, 2, 68, 77, 1, 40], [75, 31, 91, 54, 51, 72, 73, 23, 3, 70], [58, 69, 74, 50, 0, 52, 75, 51, 94, 60], [45, 70, 35, 93, 4, 43, 67, 17, 1, 86], [67, 53, 77, 95, 1, 28, 33, 47, 38, 78], [7, 66, 33, 67, 85, 59, 5, 93, 41, 29], [19, 17, 57, 70, 5, 75, 28, 47, 32, 31]], [[66, 32, 35, 20, 34, 42, 65, 33, 38, 29], [49, 52, 60, 28, 16, 89, 7, 67, 94, 21], [73, 25, 19, 9, 30, 81, 7, 21, 14, 82], [50, 80, 95, 71, 1, 49, 90, 28, 57, 52], [20, 60, 10, 47, 5, 44, 46, 21, 83, 17], [45, 73, 41, 89, 90, 50, 10, 91, 81, 64], [81, 6, 71, 14, 8, 33, 25, 55, 27, 72], [73, 57, 22, 65, 88, 68, 0, 23, 46, 42]], [[17, 23, 52, 46, 37, 48, 3, 72, 36, 61], [14, 62, 46, 37, 52, 33, 9, 42, 20, 65], [7, 54, 0, 42, 14, 57, 25, 10, 65, 23], [23, 81, 61, 86, 58, 52, 90, 15, 35, 64], [19, 52, 74, 93, 11, 3, 84, 14, 92, 27], [45, 56, 26, 15, 0, 11, 75, 51, 14, 24], [49, 47, 53, 66, 11, 5, 64, 59, 77, 50], [49, 52, 21, 37, 35, 50, 23, 15, 77, 63]], [[52, 44, 70, 14, 4, 48, 76, 63, 1, 11], [45, 21, 82, 19, 93, 75, 73, 4, 43, 91], [19, 68, 60, 84, 21, 79, 44, 65, 20, 1], [73, 49, 67, 71, 14, 40, 3, 87, 9, 8], [81, 20, 77, 15, 47, 60, 71, 5, 92, 7], [83, 87, 44, 73, 85, 60, 56, 28, 3, 33], [11, 25, 67, 93, 27, 76, 70, 61, 15, 72], [36, 49, 80, 35, 21, 22, 32, 12, 69, 71]], [[89, 16, 7, 23, 52, 38, 8, 47, 42, 18], [36, 64, 9, 79, 38, 26, 94, 66, 58, 12], [6, 40, 78, 44, 29, 43, 28, 34, 60, 24], [62, 84, 32, 7, 11, 94, 39, 91, 3, 12], [16, 63, 55, 32, 70, 87, 18, 85, 4, 86], [7, 81, 91, 44, 93, 73, 18, 38, 53, 12], [44, 60, 32, 50, 0, 46, 74, 35, 79, 83], [83, 82, 88, 23, 19, 33, 72, 41, 45, 13]], [[44, 58, 75, 52, 74, 21, 16, 23, 47, 6], [78, 77, 14, 82, 46, 40, 61, 24, 42, 81], [45, 58, 27, 60, 80, 23, 63, 34, 52, 92], [71, 16, 79, 60, 62, 11, 53, 9, 40, 31], [40, 12, 25, 26, 31, 42, 78, 1, 9, 89], [23, 51, 15, 24, 56, 39, 0, 35, 67, 4], [19, 1, 86, 59, 31, 74, 47, 20, 4, 5], [83, 53, 25, 3, 1, 81, 41, 63, 13, 88]], [[33, 75, 39, 29, 74, 58, 43, 82, 26, 73], [45, 54, 10, 75, 43, 56, 7, 68, 63, 2], [26, 65, 64, 22, 68, 80, 27, 48, 77, 9], [26, 32, 34, 46, 80, 69, 54, 31, 42, 76], [1, 41, 78, 3, 31, 88, 45, 79, 80, 49], [19, 35, 81, 51, 93, 28, 48, 63, 80, 30], [86, 36, 43, 63, 1, 60, 33, 74, 81, 57], [42, 49, 24, 46, 90, 66, 45, 87, 59, 55]], [[4, 67, 64, 24, 56, 73, 55, 43, 54, 92], [30, 2, 73, 4, 1, 34, 9, 11, 95, 29], [5, 10, 88, 42, 86, 67, 80, 25, 53, 38], [59, 66, 52, 77, 73, 35, 89, 26, 45, 72], [56, 83, 24, 73, 28, 61, 50, 93, 64, 13], [82, 73, 11, 45, 31, 41, 80, 58, 1, 5], [17, 91, 25, 2, 42, 56, 8, 40, 15, 33], [57, 84, 95, 4, 64, 83, 26, 66, 15, 30]], [[84, 66, 88, 58, 71, 15, 45, 50, 38, 44], [17, 29, 20, 91, 79, 32, 44, 66, 70, 6], [12, 16, 26, 65, 51, 67, 48, 79, 29, 95], [78, 88, 55, 27, 84, 79, 24, 54, 70, 76], [61, 81, 6, 51, 64, 40, 18, 69, 83, 16], [17, 35, 51, 42, 55, 86, 36, 24, 50, 46], [23, 73, 12, 33, 89, 26, 63, 59, 44, 65], [34, 24, 22, 64, 27, 88, 46, 70, 44, 30]]]

clss_support_imagesss_5_shot=[[[32, 30, 44, 38, 25], [82, 83, 48, 43, 90], [7, 95, 9, 14, 63], [72, 82, 69, 26, 6], [25, 67, 41, 47, 95], [44, 57, 84, 72, 58], [7, 68, 51, 47, 63], [67, 17, 31, 62, 12]], [[50, 94, 95, 27, 35], [74, 33, 78, 25, 11], [33, 56, 67, 11, 31], [71, 37, 27, 11, 65], [67, 29, 87, 60, 71], [16, 83, 67, 4, 50], [20, 87, 71, 53, 90], [14, 77, 74, 42, 27]], [[30, 48, 0, 84, 3], [93, 69, 13, 90, 78], [6, 20, 80, 56, 33], [20, 50, 8, 76, 15], [54, 46, 37, 77, 38], [50, 72, 23, 28, 85], [8, 45, 72, 36, 93], [71, 91, 30, 49, 66]], [[4, 73, 59, 63, 39], [14, 19, 31, 63, 9], [40, 94, 59, 17, 57], [29, 56, 59, 71, 78], [19, 0, 83, 93, 15], [51, 19, 38, 46, 5], [40, 32, 95, 4, 81], [47, 28, 38, 60, 59]], [[53, 40, 2, 54, 68], [23, 49, 47, 43, 7], [12, 35, 13, 79, 86], [2, 42, 85, 0, 7], [78, 72, 46, 12, 1], [9, 88, 77, 7, 84], [56, 89, 26, 69, 17], [73, 11, 22, 27, 48]], [[62, 40, 87, 85, 93], [55, 83, 60, 17, 19], [76, 79, 90, 65, 60], [65, 91, 34, 14, 10], [44, 45, 50, 35, 13], [82, 44, 14, 77, 19], [18, 90, 85, 7, 16], [74, 76, 91, 47, 63]], [[19, 26, 77, 36, 52], [23, 34, 84, 62, 4], [75, 31, 91, 54, 51], [58, 69, 74, 50, 0], [45, 70, 35, 93, 4], [67, 53, 77, 95, 1], [7, 66, 33, 67, 85], [19, 17, 57, 70, 5]], [[66, 32, 35, 20, 34], [49, 52, 60, 28, 16], [73, 25, 19, 9, 30], [50, 80, 95, 71, 1], [20, 60, 10, 47, 5], [45, 73, 41, 89, 90], [81, 6, 71, 14, 8], [73, 57, 22, 65, 88]], [[17, 23, 52, 46, 37], [14, 62, 46, 37, 52], [7, 54, 0, 42, 14], [23, 81, 61, 86, 58], [19, 52, 74, 93, 11], [45, 56, 26, 15, 0], [49, 47, 53, 66, 11], [49, 52, 21, 37, 35]], [[52, 44, 70, 14, 4], [45, 21, 82, 19, 93], [19, 68, 60, 84, 21], [73, 49, 67, 71, 14], [81, 20, 77, 15, 47], [83, 87, 44, 73, 85], [11, 25, 67, 93, 27], [36, 49, 80, 35, 21]], [[89, 16, 7, 23, 52], [36, 64, 9, 79, 38], [6, 40, 78, 44, 29], [62, 84, 32, 7, 11], [16, 63, 55, 32, 70], [7, 81, 91, 44, 93], [44, 60, 32, 50, 0], [83, 82, 88, 23, 19]], [[44, 58, 75, 52, 74], [78, 77, 14, 82, 46], [45, 58, 27, 60, 80], [71, 16, 79, 60, 62], [40, 12, 25, 26, 31], [23, 51, 15, 24, 56], [19, 1, 86, 59, 31], [83, 53, 25, 3, 1]], [[33, 75, 39, 29, 74], [45, 54, 10, 75, 43], [26, 65, 64, 22, 68], [26, 32, 34, 46, 80], [1, 41, 78, 3, 31], [19, 35, 81, 51, 93], [86, 36, 43, 63, 1], [42, 49, 24, 46, 90]], [[4, 67, 64, 24, 56], [30, 2, 73, 4, 1], [5, 10, 88, 42, 86], [59, 66, 52, 77, 73], [56, 83, 24, 73, 28], [82, 73, 11, 45, 31], [17, 91, 25, 2, 42], [57, 84, 95, 4, 64]], [[84, 66, 88, 58, 71], [17, 29, 20, 91, 79], [12, 16, 26, 65, 51], [78, 88, 55, 27, 84], [61, 81, 6, 51, 64], [17, 35, 51, 42, 55], [23, 73, 12, 33, 89], [34, 24, 22, 64, 27]]]


clss_support_imagesss_1_shot=[
    [[32], [82], [7], [72], [25], [44], [7], [67]],
    [[50], [74], [33], [71], [67], [16], [20], [14]],
    [[30], [93], [6], [20], [54], [50], [8], [71]],
     [[4], [14], [40], [29], [19], [51], [40], [47]],
    [[53], [23], [12], [2], [78], [9], [56], [73]],
    [[62], [55], [76], [65], [44], [82], [18], [74]],
    [[19], [23], [75], [58], [45], [67], [7], [19]],
     [[66], [49], [73], [50], [20], [45], [81], [73]],
    [[17], [14], [7], [23], [19], [45], [49], [49]],
    [[52], [45], [19], [73], [81], [83], [11], [36]],
    [[89], [36], [6], [62], [16], [7], [44], [83]],
    [[44], [78], [45], [71], [40], [23], [19], [83]],
    [[33], [45], [26], [26], [1], [19], [86], [42]],
    [[4], [30], [5], [59], [56], [82], [17], [57]],
    [[84], [17], [12], [78], [61], [17], [23], [34]]

]



In [356]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/mapi_sce/Denssecondary/model_1/model_B_mapi_1-shot_dens.pth',map_location=torch.device('cpu')))

torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [357]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

           
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [358]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.44868421052631574
Evaluated Accuracy: 0.4487
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.25622807017543864
Evaluated Accuracy: 0.2562
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.2025438596491228
Evaluated Accuracy: 0.2025
Testing b_a_r = 0.4559, c_a_r = 0.0270, f_a_r = 0.0256


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.48561403508771933
Evaluated Accuracy: 0.4856
Testing b_a_r = 0.3598, c_a_r = 0.0745, f_a_r = 0.1038


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.48166666666666674
Evaluated Accuracy: 0.4817

✅ Optimal Values:
   - b_a_r: 0.0329
   - c_a_r: 0.0298
   - f_a_r: 0.0373
📈 Highest Accuracy Achieved: 0.4871


In [360]:
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_mapii/result_densnet/base/8_w_1_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/secondary/8_w_1_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/amul/8_w_1_s_f", test_loader, M3, criterion)


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4854 ± 0.0475  0.5154 ± 0.0489  0.4854 ± 0.0475


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.3607 ± 0.0488  0.3991 ± 0.0597  0.3607 ± 0.0488
---Accu-----pre----rec---------> 0.4871 ± 0.0452  0.5183 ± 0.0475  0.4871 ± 0.0452


In [444]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/mapi_sce/Denssecondary/model_5/model_B_mapi_5-shot_dens.pth',map_location=torch.device('cpu')))

torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [445]:
def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

           
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

            
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [446]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5392857142857143
Evaluated Accuracy: 0.5393
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.2888278388278388
Evaluated Accuracy: 0.2888
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.2756410256410256
Evaluated Accuracy: 0.2756
Testing b_a_r = 0.9902, c_a_r = 0.0869, f_a_r = 0.9800


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5810439560439561
Evaluated Accuracy: 0.5810
Testing b_a_r = 0.0440, c_a_r = 0.0006, f_a_r = 0.0998


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5941391941391941
Evaluated Accuracy: 0.5941

✅ Optimal Values:
   - b_a_r: 0.0000
   - c_a_r: 0.0000
   - f_a_r: 0.0000
📈 Highest Accuracy Achieved: 0.5946


In [447]:
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_mapii/result_densnet/base/8_w_5_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/secondary/8_w_5_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/amul/8_w_5_s_f", test_loader, M3, criterion)


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5946 ± 0.0447  0.6063 ± 0.0525  0.5946 ± 0.0447


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5039 ± 0.0622  0.5150 ± 0.0657  0.5039 ± 0.0622
---Accu-----pre----rec---------> 0.5946 ± 0.0447  0.6063 ± 0.0525  0.5946 ± 0.0447


In [99]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/mapi_sce/Denssecondary/model_10/model_B_mapi_10-shot_dens.pth',map_location=torch.device('cpu')))


torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [100]:
def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

           
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

            
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [101]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=80
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5929263565891474
Evaluated Accuracy: 0.5929
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.2684108527131783
Evaluated Accuracy: 0.2684
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.39699612403100776
Evaluated Accuracy: 0.3970
Testing b_a_r = 0.6331, c_a_r = 0.4078, f_a_r = 0.2183


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.3373062015503876
Evaluated Accuracy: 0.3373
Testing b_a_r = 0.0282, c_a_r = 0.0418, f_a_r = 0.0048


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6245155038759691
Evaluated Accuracy: 0.6245

✅ Optimal Values:
   - b_a_r: 0.0242
   - c_a_r: 0.1140
   - f_a_r: 0.9980
📈 Highest Accuracy Achieved: 0.6360


In [102]:
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_mapii/result_densnet/base/8_w_10_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/secondary/8_w_10_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/amul/8_w_10_s_f", test_loader, M3, criterion)


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.6192 ± 0.0423  0.6277 ± 0.0464  0.6192 ± 0.0423


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5266 ± 0.0447  0.5296 ± 0.0486  0.5266 ± 0.0447
---Accu-----pre----rec---------> 0.6360 ± 0.0360  0.6469 ± 0.0369  0.6360 ± 0.0360


In [103]:
###############################################   12 way 1 shot

In [104]:
# clss_20=
# clss_1_shot_20_w=
# clss_5_shot_20_w=
# clss_10_shot_20_w=

In [405]:
clss_support_imagesss_10_shot=[[[21, 14, 23, 29, 67, 54, 10, 91, 61, 52], [40, 71, 6, 67, 65, 37, 0, 74, 11, 83], [89, 58, 33, 13, 23, 88, 77, 62, 18, 34], [6, 24, 74, 14, 37, 71, 72, 75, 91, 38], [10, 35, 15, 91, 71, 74, 86, 12, 80, 23], [93, 24, 81, 78, 83, 49, 15, 85, 10, 25], [32, 73, 51, 14, 42, 87, 74, 77, 70, 61], [36, 0, 34, 82, 39, 31, 1, 52, 27, 50], [89, 55, 56, 32, 80, 58, 53, 76, 63, 2], [22, 14, 80, 1, 6, 69, 50, 49, 72, 10], [37, 9, 46, 70, 12, 84, 23, 25, 75, 36], [7, 81, 29, 40, 46, 8, 37, 57, 52, 2]], [[66, 39, 90, 45, 52, 93, 58, 20, 12, 95], [63, 61, 91, 49, 5, 53, 56, 15, 57, 62], [93, 72, 86, 79, 27, 18, 8, 23, 22, 54], [67, 3, 58, 84, 72, 30, 1, 32, 94, 43], [15, 86, 95, 33, 54, 94, 28, 12, 93, 40], [94, 91, 48, 21, 86, 35, 76, 66, 32, 70], [46, 24, 71, 61, 64, 88, 63, 39, 70, 44], [34, 48, 62, 63, 64, 82, 57, 15, 47, 65], [58, 60, 72, 49, 52, 73, 71, 79, 33, 44], [39, 64, 28, 15, 20, 78, 50, 38, 59, 86], [26, 78, 67, 34, 92, 40, 48, 90, 53, 8], [39, 42, 64, 40, 32, 46, 95, 7, 85, 44]], [[64, 80, 7, 65, 40, 5, 13, 49, 58, 19], [71, 66, 67, 84, 64, 19, 5, 82, 33, 4], [38, 82, 13, 91, 14, 34, 67, 37, 58, 85], [26, 64, 89, 84, 95, 41, 66, 2, 77, 8], [17, 83, 77, 37, 39, 25, 86, 92, 79, 22], [52, 2, 60, 37, 73, 17, 89, 8, 4, 64], [71, 9, 59, 87, 49, 31, 4, 79, 22, 0], [21, 6, 79, 86, 74, 34, 19, 82, 62, 33], [45, 36, 26, 57, 22, 93, 3, 27, 2, 89], [4, 71, 51, 25, 84, 24, 10, 5, 85, 27], [29, 61, 79, 82, 18, 40, 70, 26, 32, 68], [56, 63, 64, 66, 70, 45, 87, 48, 88, 18]], [[64, 80, 50, 62, 0, 8, 58, 34, 91, 52], [39, 8, 57, 52, 44, 91, 80, 69, 60, 81], [36, 27, 19, 54, 11, 95, 71, 80, 9, 51], [63, 70, 2, 59, 10, 89, 43, 37, 27, 21], [26, 50, 86, 12, 47, 30, 52, 32, 7, 67], [17, 29, 94, 6, 50, 28, 43, 25, 56, 70], [31, 39, 56, 32, 43, 76, 71, 19, 49, 27], [2, 3, 64, 92, 57, 46, 6, 67, 78, 10], [29, 84, 78, 36, 59, 14, 41, 42, 0, 90], [63, 2, 13, 39, 16, 1, 14, 31, 77, 44], [9, 14, 6, 60, 95, 61, 19, 40, 20, 39], [7, 3, 68, 59, 11, 5, 61, 21, 1, 14]], [[48, 20, 92, 33, 18, 91, 35, 52, 41, 86], [59, 8, 20, 88, 73, 72, 94, 11, 28, 66], [62, 28, 50, 93, 79, 95, 88, 10, 14, 80], [38, 94, 71, 34, 27, 18, 64, 79, 15, 11], [74, 94, 13, 10, 12, 24, 70, 48, 41, 50], [7, 10, 83, 95, 54, 71, 13, 40, 4, 60], [0, 47, 66, 61, 65, 60, 24, 43, 83, 63], [65, 51, 45, 63, 75, 10, 83, 42, 27, 58], [70, 11, 61, 8, 0, 6, 25, 31, 24, 62], [72, 68, 43, 4, 88, 22, 92, 41, 27, 91], [56, 92, 14, 34, 9, 21, 68, 76, 49, 4], [68, 49, 17, 32, 5, 90, 26, 13, 60, 69]], [[55, 74, 21, 10, 8, 88, 31, 53, 92, 59], [40, 63, 76, 34, 13, 31, 43, 48, 66, 18], [74, 3, 8, 34, 60, 47, 78, 50, 27, 77], [29, 20, 16, 53, 52, 77, 14, 47, 2, 94], [45, 88, 59, 54, 77, 55, 90, 84, 32, 65], [6, 87, 81, 36, 95, 88, 73, 21, 57, 67], [58, 43, 30, 94, 15, 49, 62, 53, 31, 87], [78, 6, 85, 70, 33, 77, 24, 89, 29, 1], [41, 69, 86, 23, 17, 43, 94, 32, 56, 8], [95, 80, 52, 35, 43, 22, 29, 67, 82, 63], [41, 73, 27, 28, 78, 19, 23, 69, 13, 68], [2, 12, 80, 42, 86, 28, 14, 70, 39, 91]], [[76, 3, 44, 35, 67, 32, 63, 27, 26, 68], [75, 63, 0, 71, 23, 45, 36, 81, 69, 72], [44, 19, 88, 73, 83, 90, 33, 45, 93, 11], [3, 27, 4, 48, 74, 50, 78, 80, 18, 95], [35, 78, 45, 70, 43, 65, 0, 11, 28, 37], [10, 8, 60, 84, 12, 36, 88, 41, 31, 46], [26, 54, 82, 20, 41, 51, 48, 92, 31, 49], [39, 82, 95, 38, 12, 15, 63, 83, 21, 90], [61, 48, 54, 83, 46, 23, 44, 22, 38, 32], [27, 92, 58, 35, 76, 8, 89, 23, 83, 91], [84, 7, 5, 83, 18, 3, 76, 21, 44, 20], [70, 59, 37, 4, 56, 80, 84, 76, 50, 38]], [[3, 7, 56, 39, 64, 92, 12, 21, 55, 20], [34, 81, 92, 86, 30, 93, 82, 29, 57, 63], [52, 36, 19, 14, 10, 1, 25, 17, 29, 34], [14, 62, 69, 59, 3, 28, 36, 79, 67, 7], [89, 60, 16, 44, 86, 56, 39, 12, 20, 64], [45, 73, 50, 80, 60, 41, 93, 86, 24, 49], [89, 70, 24, 16, 26, 23, 35, 12, 41, 72], [45, 0, 34, 59, 51, 6, 12, 66, 25, 35], [7, 44, 73, 38, 78, 89, 11, 87, 4, 80], [0, 33, 63, 12, 56, 23, 60, 21, 47, 14], [37, 66, 42, 26, 7, 63, 52, 50, 13, 19], [10, 49, 85, 47, 78, 27, 3, 48, 53, 15]], [[36, 69, 29, 48, 68, 42, 18, 49, 78, 50], [1, 92, 9, 75, 87, 88, 25, 50, 32, 39], [72, 2, 17, 41, 15, 16, 26, 0, 14, 51], [29, 90, 48, 92, 79, 40, 52, 88, 35, 7], [73, 6, 79, 14, 29, 33, 91, 76, 10, 36], [56, 64, 85, 67, 9, 31, 93, 60, 58, 5], [27, 23, 88, 28, 26, 92, 1, 84, 70, 21], [26, 48, 40, 54, 95, 83, 60, 62, 33, 44], [37, 24, 26, 25, 69, 58, 80, 18, 78, 71], [25, 68, 88, 1, 87, 53, 36, 35, 45, 30], [32, 78, 30, 9, 80, 62, 77, 28, 13, 89], [63, 1, 26, 30, 3, 64, 67, 70, 29, 76]], [[65, 53, 94, 51, 89, 32, 26, 38, 62, 61], [95, 84, 72, 74, 79, 30, 47, 17, 15, 64], [90, 67, 6, 14, 81, 34, 50, 26, 86, 17], [70, 59, 21, 3, 77, 62, 63, 68, 34, 90], [3, 84, 46, 4, 5, 2, 12, 89, 74, 45], [92, 74, 95, 54, 60, 86, 83, 43, 59, 78], [2, 36, 29, 67, 22, 94, 78, 16, 30, 74], [63, 53, 89, 23, 86, 35, 57, 49, 66, 21], [14, 54, 48, 58, 76, 25, 63, 30, 24, 78], [52, 32, 1, 95, 54, 57, 16, 3, 89, 14], [11, 67, 66, 33, 2, 72, 10, 4, 79, 5], [71, 1, 42, 75, 60, 84, 16, 88, 4, 19]], [[32, 70, 12, 11, 93, 36, 68, 69, 23, 78], [44, 83, 87, 53, 94, 31, 66, 95, 65, 55], [32, 46, 89, 41, 62, 81, 52, 92, 1, 47], [25, 7, 24, 85, 81, 93, 58, 72, 34, 67], [64, 74, 12, 41, 11, 52, 48, 26, 93, 34], [33, 58, 64, 56, 38, 37, 61, 14, 3, 50], [22, 93, 23, 70, 64, 63, 5, 40, 33, 9], [86, 21, 93, 1, 26, 32, 18, 81, 52, 10], [63, 89, 24, 45, 1, 51, 36, 6, 95, 57], [83, 13, 40, 31, 92, 88, 82, 51, 95, 52], [86, 62, 33, 31, 72, 18, 74, 12, 0, 68], [11, 20, 70, 34, 64, 25, 30, 9, 46, 18]], [[43, 76, 1, 61, 59, 74, 49, 91, 87, 94], [73, 26, 81, 83, 47, 74, 15, 45, 12, 16], [30, 34, 26, 47, 28, 94, 57, 4, 35, 68], [89, 13, 40, 58, 73, 12, 72, 0, 85, 94], [17, 2, 89, 75, 41, 95, 59, 53, 84, 86], [81, 57, 20, 72, 6, 14, 85, 64, 41, 37], [49, 30, 6, 48, 58, 31, 5, 20, 67, 85], [51, 74, 16, 14, 34, 84, 79, 75, 77, 30], [23, 51, 33, 83, 79, 81, 53, 61, 49, 84], [6, 66, 51, 95, 4, 59, 85, 71, 69, 2], [40, 4, 26, 87, 56, 17, 71, 57, 13, 85], [57, 41, 15, 67, 83, 87, 44, 93, 12, 30]], [[4, 21, 66, 88, 71, 57, 18, 16, 73, 41], [69, 60, 38, 32, 92, 53, 54, 47, 2, 27], [40, 17, 59, 39, 74, 18, 38, 23, 95, 2], [21, 29, 76, 74, 9, 68, 32, 35, 69, 51], [70, 60, 13, 49, 94, 14, 24, 8, 73, 67], [86, 32, 54, 13, 58, 87, 27, 92, 24, 83], [21, 41, 24, 56, 88, 27, 30, 11, 71, 67], [15, 12, 86, 91, 82, 38, 58, 76, 2, 33], [45, 81, 95, 38, 43, 79, 27, 61, 64, 87], [66, 1, 90, 77, 85, 15, 50, 53, 87, 21], [92, 39, 7, 50, 89, 36, 81, 68, 52, 23], [5, 87, 49, 55, 57, 66, 18, 37, 79, 50]], [[22, 32, 91, 72, 81, 69, 8, 34, 15, 20], [89, 31, 91, 27, 40, 90, 72, 19, 4, 9], [5, 60, 51, 34, 8, 56, 88, 52, 78, 69], [86, 5, 9, 50, 18, 45, 93, 28, 89, 30], [26, 35, 19, 90, 58, 10, 79, 52, 92, 45], [17, 91, 77, 33, 5, 40, 64, 89, 24, 60], [93, 77, 36, 60, 13, 74, 75, 71, 45, 14], [48, 42, 78, 58, 29, 36, 65, 74, 4, 85], [33, 81, 90, 38, 88, 66, 34, 49, 26, 93], [30, 12, 36, 7, 15, 6, 0, 13, 25, 43], [8, 86, 52, 10, 32, 4, 69, 6, 79, 65], [37, 2, 41, 15, 45, 36, 76, 32, 3, 22]], [[50, 81, 36, 1, 39, 63, 90, 72, 58, 77], [0, 30, 38, 83, 13, 14, 59, 90, 23, 22], [84, 60, 45, 23, 0, 48, 32, 18, 24, 29], [92, 19, 29, 89, 25, 75, 53, 24, 45, 66], [90, 92, 53, 85, 0, 23, 70, 88, 51, 39], [75, 14, 70, 24, 10, 2, 66, 28, 18, 68], [67, 52, 46, 45, 60, 34, 59, 91, 21, 77], [29, 2, 94, 65, 75, 87, 33, 25, 55, 6], [60, 34, 75, 74, 18, 20, 31, 10, 22, 83], [18, 77, 61, 24, 53, 60, 54, 28, 39, 21], [55, 87, 43, 12, 83, 57, 10, 23, 90, 93], [33, 32, 13, 52, 16, 54, 84, 91, 81, 38]]]



clss_support_imagesss_5_shot=[[[21, 14, 23, 29, 67], [40, 71, 6, 67, 65], [89, 58, 33, 13, 23], [6, 24, 74, 14, 37], [10, 35, 15, 91, 71], [93, 24, 81, 78, 83], [32, 73, 51, 14, 42], [36, 0, 34, 82, 39], [89, 55, 56, 32, 80], [22, 14, 80, 1, 6], [37, 9, 46, 70, 12], [7, 81, 29, 40, 46]], [[66, 39, 90, 45, 52], [63, 61, 91, 49, 5], [93, 72, 86, 79, 27], [67, 3, 58, 84, 72], [15, 86, 95, 33, 54], [94, 91, 48, 21, 86], [46, 24, 71, 61, 64], [34, 48, 62, 63, 64], [58, 60, 72, 49, 52], [39, 64, 28, 15, 20], [26, 78, 67, 34, 92], [39, 42, 64, 40, 32]], [[64, 80, 7, 65, 40], [71, 66, 67, 84, 64], [38, 82, 13, 91, 14], [26, 64, 89, 84, 95], [17, 83, 77, 37, 39], [52, 2, 60, 37, 73], [71, 9, 59, 87, 49], [21, 6, 79, 86, 74], [45, 36, 26, 57, 22], [4, 71, 51, 25, 84], [29, 61, 79, 82, 18], [56, 63, 64, 66, 70]], [[64, 80, 50, 62, 0], [39, 8, 57, 52, 44], [36, 27, 19, 54, 11], [63, 70, 2, 59, 10], [26, 50, 86, 12, 47], [17, 29, 94, 6, 50], [31, 39, 56, 32, 43], [2, 3, 64, 92, 57], [29, 84, 78, 36, 59], [63, 2, 13, 39, 16], [9, 14, 6, 60, 95], [7, 3, 68, 59, 11]], [[48, 20, 92, 33, 18], [59, 8, 20, 88, 73], [62, 28, 50, 93, 79], [38, 94, 71, 34, 27], [74, 94, 13, 10, 12], [7, 10, 83, 95, 54], [0, 47, 66, 61, 65], [65, 51, 45, 63, 75], [70, 11, 61, 8, 0], [72, 68, 43, 4, 88], [56, 92, 14, 34, 9], [68, 49, 17, 32, 5]], [[55, 74, 21, 10, 8], [40, 63, 76, 34, 13], [74, 3, 8, 34, 60], [29, 20, 16, 53, 52], [45, 88, 59, 54, 77], [6, 87, 81, 36, 95], [58, 43, 30, 94, 15], [78, 6, 85, 70, 33], [41, 69, 86, 23, 17], [95, 80, 52, 35, 43], [41, 73, 27, 28, 78], [2, 12, 80, 42, 86]], [[76, 3, 44, 35, 67], [75, 63, 0, 71, 23], [44, 19, 88, 73, 83], [3, 27, 4, 48, 74], [35, 78, 45, 70, 43], [10, 8, 60, 84, 12], [26, 54, 82, 20, 41], [39, 82, 95, 38, 12], [61, 48, 54, 83, 46], [27, 92, 58, 35, 76], [84, 7, 5, 83, 18], [70, 59, 37, 4, 56]], [[3, 7, 56, 39, 64], [34, 81, 92, 86, 30], [52, 36, 19, 14, 10], [14, 62, 69, 59, 3], [89, 60, 16, 44, 86], [45, 73, 50, 80, 60], [89, 70, 24, 16, 26], [45, 0, 34, 59, 51], [7, 44, 73, 38, 78], [0, 33, 63, 12, 56], [37, 66, 42, 26, 7], [10, 49, 85, 47, 78]], [[36, 69, 29, 48, 68], [1, 92, 9, 75, 87], [72, 2, 17, 41, 15], [29, 90, 48, 92, 79], [73, 6, 79, 14, 29], [56, 64, 85, 67, 9], [27, 23, 88, 28, 26], [26, 48, 40, 54, 95], [37, 24, 26, 25, 69], [25, 68, 88, 1, 87], [32, 78, 30, 9, 80], [63, 1, 26, 30, 3]], [[65, 53, 94, 51, 89], [95, 84, 72, 74, 79], [90, 67, 6, 14, 81], [70, 59, 21, 3, 77], [3, 84, 46, 4, 5], [92, 74, 95, 54, 60], [2, 36, 29, 67, 22], [63, 53, 89, 23, 86], [14, 54, 48, 58, 76], [52, 32, 1, 95, 54], [11, 67, 66, 33, 2], [71, 1, 42, 75, 60]], [[32, 70, 12, 11, 93], [44, 83, 87, 53, 94], [32, 46, 89, 41, 62], [25, 7, 24, 85, 81], [64, 74, 12, 41, 11], [33, 58, 64, 56, 38], [22, 93, 23, 70, 64], [86, 21, 93, 1, 26], [63, 89, 24, 45, 1], [83, 13, 40, 31, 92], [86, 62, 33, 31, 72], [11, 20, 70, 34, 64]], [[43, 76, 1, 61, 59], [73, 26, 81, 83, 47], [30, 34, 26, 47, 28], [89, 13, 40, 58, 73], [17, 2, 89, 75, 41], [81, 57, 20, 72, 6], [49, 30, 6, 48, 58], [51, 74, 16, 14, 34], [23, 51, 33, 83, 79], [6, 66, 51, 95, 4], [40, 4, 26, 87, 56], [57, 41, 15, 67, 83]], [[4, 21, 66, 88, 71], [69, 60, 38, 32, 92], [40, 17, 59, 39, 74], [21, 29, 76, 74, 9], [70, 60, 13, 49, 94], [86, 32, 54, 13, 58], [21, 41, 24, 56, 88], [15, 12, 86, 91, 82], [45, 81, 95, 38, 43], [66, 1, 90, 77, 85], [92, 39, 7, 50, 89], [5, 87, 49, 55, 57]], [[22, 32, 91, 72, 81], [89, 31, 91, 27, 40], [5, 60, 51, 34, 8], [86, 5, 9, 50, 18], [26, 35, 19, 90, 58], [17, 91, 77, 33, 5], [93, 77, 36, 60, 13], [48, 42, 78, 58, 29], [33, 81, 90, 38, 88], [30, 12, 36, 7, 15], [8, 86, 52, 10, 32], [37, 2, 41, 15, 45]], [[50, 81, 36, 1, 39], [0, 30, 38, 83, 13], [84, 60, 45, 23, 0], [92, 19, 29, 89, 25], [90, 92, 53, 85, 0], [75, 14, 70, 24, 10], [67, 52, 46, 45, 60], [29, 2, 94, 65, 75], [60, 34, 75, 74, 18], [18, 77, 61, 24, 53], [55, 87, 43, 12, 83], [33, 32, 13, 52, 16]]]


clss_12=[
   ['MAPI Mayeek_GOK', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_JIL', 'MAPI Mayeek_UUN', 'MAPI Mayeek_BAA', 'MAPI Mayeek_FAM', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_KOK', 'MAPI Mayeek_LAI', 'MAPI Mayeek_DIL', 'MAPI Mayeek_SAM', 'MAPI Mayeek_ATIYA'],
   ['MAPI Mayeek_JIL', 'MAPI Mayeek_EEE', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_MIT', 'MAPI Mayeek_ATIYA', 'MAPI Mayeek_BAA', 'MAPI Mayeek_HUK', 'MAPI Mayeek_THOU', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_UUN', 'MAPI Mayeek_PAA', 'MAPI Mayeek_JHAM'],
     ['MAPI Mayeek_PAA', 'MAPI Mayeek_KOK', 'MAPI Mayeek_BAA', 'MAPI Mayeek_DIL', 'MAPI Mayeek_CHIN', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_NAA', 'MAPI Mayeek_RAAI', 'MAPI Mayeek_WAI', 'MAPI Mayeek_UUN', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_LAI'],
   ['MAPI Mayeek_KHOU', 'MAPI Mayeek_FAM', 'MAPI Mayeek_SAM', 'MAPI Mayeek_JIL', 'MAPI Mayeek_EEE', 'MAPI Mayeek_LAI', 'MAPI Mayeek_HUK', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_GOK', 'MAPI Mayeek_TIL', 'MAPI Mayeek_KOK'],
    ['MAPI Mayeek_CHIN', 'MAPI Mayeek_YANG', 'MAPI Mayeek_KOK', 'MAPI Mayeek_PAA', 'MAPI Mayeek_NAA', 'MAPI Mayeek_FAM', 'MAPI Mayeek_RAAI', 'MAPI Mayeek_THOU', 'MAPI Mayeek_EEE', 'MAPI Mayeek_TIL', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_WAI'],
    ['MAPI Mayeek_NAA', 'MAPI Mayeek_EEE', 'MAPI Mayeek_KOK', 'MAPI Mayeek_BAA', 'MAPI Mayeek_WAI', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_PAA', 'MAPI Mayeek_MIT', 'MAPI Mayeek_THOU', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_NGOU'],
   ['MAPI Mayeek_NGOU', 'MAPI Mayeek_LAI', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_JIL', 'MAPI Mayeek_SAM', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_EEE', 'MAPI Mayeek_ATIYA', 'MAPI Mayeek_RAAI', 'MAPI Mayeek_MIT', 'MAPI Mayeek_KOK', 'MAPI Mayeek_FAM'],
    ['MAPI Mayeek_SAM', 'MAPI Mayeek_LAI', 'MAPI Mayeek_CHIN', 'MAPI Mayeek_THOU', 'MAPI Mayeek_PAA', 'MAPI Mayeek_KOK', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_NAA', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_MIT', 'MAPI Mayeek_HUK', 'MAPI Mayeek_JHAM'],
   ['MAPI Mayeek_TIL', 'MAPI Mayeek_WAI', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_HUK', 'MAPI Mayeek_DIL', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_FAM', 'MAPI Mayeek_LAI', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_GOK'],
    ['MAPI Mayeek_BAA', 'MAPI Mayeek_GOK', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_LAI', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_TIL', 'MAPI Mayeek_THOU', 'MAPI Mayeek_KOK', 'MAPI Mayeek_FAM', 'MAPI Mayeek_RAAI', 'MAPI Mayeek_SAM'],
   ['MAPI Mayeek_ATIYA', 'MAPI Mayeek_UUN', 'MAPI Mayeek_PAA', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_NAA', 'MAPI Mayeek_FAM', 'MAPI Mayeek_LAI', 'MAPI Mayeek_KOK', 'MAPI Mayeek_GOK', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_SAM'],
    ['MAPI Mayeek_BHAM', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_BAA', 'MAPI Mayeek_EEE', 'MAPI Mayeek_CHIN', 'MAPI Mayeek_LAI', 'MAPI Mayeek_FAM', 'MAPI Mayeek_NAA', 'MAPI Mayeek_GOK', 'MAPI Mayeek_THOU', 'MAPI Mayeek_SAM', 'MAPI Mayeek_ATIYA'],
    ['MAPI Mayeek_BHAM', 'MAPI Mayeek_LAI', 'MAPI Mayeek_PAA', 'MAPI Mayeek_NAA', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_SAM', 'MAPI Mayeek_DIL', 'MAPI Mayeek_CHIN', 'MAPI Mayeek_JHAM', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_EEE', 'MAPI Mayeek_GOK'],
   ['MAPI Mayeek_FAM', 'MAPI Mayeek_BHAM', 'MAPI Mayeek_UUN', 'MAPI Mayeek_WAI', 'MAPI Mayeek_PAA', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_KOK', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_GOK', 'MAPI Mayeek_EEE', 'MAPI Mayeek_YANG', 'MAPI Mayeek_JHAM'],
    ['MAPI Mayeek_PAA', 'MAPI Mayeek_BAA', 'MAPI Mayeek_SAM', 'MAPI Mayeek_HUK', 'MAPI Mayeek_NGOU', 'MAPI Mayeek_FAM', 'MAPI Mayeek_LAI', 'MAPI Mayeek_GHOU', 'MAPI Mayeek_DHOU', 'MAPI Mayeek_GOK', 'MAPI Mayeek_NAA', 'MAPI Mayeek_EEE'],
   ]
clss_support_imagesss_1_shot=[
 [[21], [40], [89], [6], [10], [93], [32], [36], [89], [22], [37], [7]],
 [[66], [63], [93], [67], [15], [94], [46], [34], [58], [39], [26], [39]],
 [[64], [71], [38], [26], [17], [52], [71], [21], [45], [4], [29], [56]],
 [[64], [39], [36], [63], [26], [17], [31], [2], [29], [63], [9], [7]],
 [[48], [59], [62], [38], [74], [7], [0], [65], [70], [72], [56], [68]],
 [[55], [40], [74], [29], [45], [6], [58], [78], [41], [95], [41], [2]],
 [[76], [75], [44], [3], [35], [10], [26], [39], [61], [27], [84], [70]],
 [[3], [34], [52], [14], [89], [45], [89], [45], [7], [0], [37], [10]],
 [[36], [1], [72], [29], [73], [56], [27], [26], [37], [25], [32], [63]],
 [[65], [95], [90], [70], [3], [92], [2], [63], [14], [52], [11], [71]],
 [[32], [44], [32], [25], [64], [33], [22], [86], [63], [83], [86], [11]],
  [[43], [73], [30], [89], [17], [81], [49], [51], [23], [6], [40], [57]],
 [[4], [69], [40], [21], [70], [86], [21], [15], [45], [66], [92], [5]],
 [[22], [89], [5], [86], [26], [17], [93], [48], [33], [30], [8], [37]],
 [[50], [0], [84], [92], [90], [75], [67], [29], [60], [18], [55], [33]]
]











In [438]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/mapi_sce/Denssecondary/model_1/model_B_mapi_1-shot_dens.pth',map_location=torch.device('cpu')))


torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')


rm: cannot remove 'model_mu_path.pt': No such file or directory
rm: cannot remove 'model_own_path.pt': No such file or directory


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [439]:

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

           
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [436]:
2+3

5

In [440]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=50,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.1292, c_a_r = 0.0744, f_a_r = 0.6099


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.3592397660818713
Evaluated Accuracy: 0.3592
Testing b_a_r = 0.0214, c_a_r = 0.1522, f_a_r = 0.9547


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.3323976608187134
Evaluated Accuracy: 0.3324
Testing b_a_r = 0.8047, c_a_r = 0.2442, f_a_r = 0.3252


...

----------------------> 0.16625730994152044
Evaluated Accuracy: 0.1663
Testing b_a_r = 0.0846, c_a_r = 0.0021, f_a_r = 0.9868


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.3722222222222222
Evaluated Accuracy: 0.3722
Testing b_a_r = 0.0009, c_a_r = 0.0021, f_a_r = 0.0927


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.3726315789473684
Evaluated Accuracy: 0.3726

✅ Optimal Values:
   - b_a_r: 0.0242
   - c_a_r: 0.0001
   - f_a_r: 0.0498
📈 Highest Accuracy Achieved: 0.3728


In [442]:
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_mapii/result_densnet/base/12_w_1_s_f_20', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/secondary/12_w_1_s_f_20", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/amul/12_w_1_s_f_20", test_loader, M3, criterion)


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.3730 ± 0.0610  0.4269 ± 0.0657  0.3730 ± 0.0610


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.3679 ± 0.0553  0.4222 ± 0.0582  0.3679 ± 0.0553
---Accu-----pre----rec---------> 0.3728 ± 0.0610  0.4266 ± 0.0656  0.3728 ± 0.0610


In [419]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/mapi_sce/Denssecondary/model_5/model_B_mapi_5-shot_dens.pth',map_location=torch.device('cpu')))


torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

rm: cannot remove 'model_mu_path.pt': No such file or directory
rm: cannot remove 'model_own_path.pt': No such file or directory


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [420]:

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

           
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

            
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [423]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.4427350427350428
Evaluated Accuracy: 0.4427
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.2240537240537241
Evaluated Accuracy: 0.2241
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.21984126984126987
Evaluated Accuracy: 0.2198
Testing b_a_r = 0.8852, c_a_r = 0.6298, f_a_r = 0.7670


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.18095238095238095
Evaluated Accuracy: 0.1810
Testing b_a_r = 0.3598, c_a_r = 0.0745, f_a_r = 0.1038


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.4919413919413919
Evaluated Accuracy: 0.4919

✅ Optimal Values:
   - b_a_r: 0.0214
   - c_a_r: 0.0379
   - f_a_r: 0.0051
📈 Highest Accuracy Achieved: 0.4980


In [424]:
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_mapii/result_densnet/base/12_w_5_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/secondary/12_w_5_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/amul/12_w_5_s_f", test_loader, M3, criterion)


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4977 ± 0.0540  0.5114 ± 0.0610  0.4977 ± 0.0540


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4363 ± 0.0525  0.4407 ± 0.0542  0.4363 ± 0.0525
---Accu-----pre----rec---------> 0.4980 ± 0.0547  0.5131 ± 0.0619  0.4980 ± 0.0547


In [115]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/mapi_sce/Denssecondary/model_10/model_B_mapi_10-shot_dens.pth',map_location=torch.device('cpu')))



torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [116]:

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

           
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

            
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [117]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.4945090439276486
Evaluated Accuracy: 0.4945
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.21427648578811367
Evaluated Accuracy: 0.2143
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.44211886304909565
Evaluated Accuracy: 0.4421
Testing b_a_r = 0.9920, c_a_r = 0.1072, f_a_r = 0.0891


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5370801033591731
Evaluated Accuracy: 0.5371
Testing b_a_r = 0.0202, c_a_r = 0.0126, f_a_r = 0.9423


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5249354005167959
Evaluated Accuracy: 0.5249

✅ Optimal Values:
   - b_a_r: 0.0022
   - c_a_r: 0.1291
   - f_a_r: 0.0298
📈 Highest Accuracy Achieved: 0.5423


In [118]:
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_mapii/result_densnet/base/12_w_10_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout =  DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()

xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/secondary/12_w_10_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_mapii/result_densnet/amul/12_w_10_s_f", test_loader, M3, criterion)


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5237 ± 0.0494  0.5307 ± 0.0552  0.5237 ± 0.0494


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4271 ± 0.0479  0.4360 ± 0.0520  0.4271 ± 0.0479
---Accu-----pre----rec---------> 0.5423 ± 0.0490  0.5485 ± 0.0539  0.5423 ± 0.0490
